In [1]:
import mysql.connector
 
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Yankejing020711"   #REPLACE THIS WITH THE PASSWORD YOU SET
)
 
print(mydb)
 
if mydb.is_connected():
    print("CONNECTION SUCCESSFUL")

CONNECTION SUCCESSFUL


In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Yankejing020711", #REPLACE THIS WITH YOUR PASSWORD 
  database = "data1050f24" #connecting to testDatabase
)

1. Find the names of all students who have taken or are currently taking (in Fall' 23) Data 1050.

In [4]:
mycursor = mydb.cursor()
mycursor.execute(
    '''
    SELECT S.name
    FROM student AS S
    JOIN takes AS T
    ON S.ID = T.ID
    WHERE T.course_id = 'Data1050' AND T.semester = 'Fall' AND T.year = 2023;
    '''
)

for x in mycursor:
    print(x)

('Julia Rodriguez',)
('Jane Chen',)
('Chuning Liu',)
('Boris Pasternak',)
('Stan Ambani',)
('Peter Lynch',)
('Sabrina Zhou',)
('Maya Abdul',)
('Steve Brown',)
('Jane Lynch',)


2. Find the names of students who have taken Data 1050 more than once. (This would also include students who are currently taking Data 1050 and have also taken it in a previous semester.)

In [5]:
mycursor = mydb.cursor()
mycursor.execute(
    '''
    SELECT S.name, COUNT(S.name) AS count_1050
    FROM student AS S
    JOIN takes AS T
    ON S.ID = T.ID
    WHERE T.course_id = 'Data1050'
    GROUP BY S.name
    HAVING count_1050 > 1;
    '''
)

for x in mycursor:
    print(x)

3. For each department, find the total number of students who have taken classes offered by that department in Fall, 22. Do not include any department for which this total number is less than 2.

In [4]:
mycursor = mydb.cursor()
mycursor.execute(
    '''
    SELECT C.dept_name, COUNT(DISTINCT T.ID) AS student_count
    FROM course C
    JOIN section S ON S.course_id = C.course_id
    JOIN takes T ON T.course_id = S.course_id AND T.sec_id = S.sec_id AND T.semester = S.semester AND T.year = S.year
    WHERE S.semester = 'Fall' AND S.year = 2022
    GROUP BY C.dept_name
    HAVING student_count > 1
    '''
)

for x in mycursor:
    print(x)

('DSI', 2)


4. For each instructor,  output the name of the instructor and the total number of students they have taught in any section of any course. That is, the total number of students they have ever taught.

In [3]:
mycursor = mydb.cursor()
mycursor.execute(
    '''
    SELECT I.name, COUNT(DISTINCT TA.ID)
    FROM instructor AS I
    JOIN teaches AS T ON I.ID = T.ID
    JOIN takes AS TA ON TA.course_id = T.course_id AND TA.sec_id = T.sec_id AND TA.semester = T.semester AND TA.year = T.year
    GROUP BY I.name
    '''
)

for x in mycursor:
    print(x)

('Calista Holistic', 2)
('Carmen Herman', 2)
('Joe Smith', 11)
('Sunil Agarwal', 1)
('Ying Zheng', 5)


5. Find the id and names of instructors who have taught the least number of students.

In [5]:
mycursor = mydb.cursor()
mycursor.execute(
    '''
    SELECT I.ID, I.name
    FROM instructor AS I
    JOIN teaches AS T ON I.ID = T.ID
    JOIN takes AS TA ON TA.course_id = T.course_id AND TA.sec_id = T.sec_id AND TA.semester = T.semester AND TA.year = T.year
    GROUP BY I.ID, I.name
    HAVING COUNT(TA.ID) = (
        SELECT MIN(student_count)
        FROM (
            SELECT I.ID, COUNT(TA.ID) AS student_count
            FROM instructor AS I
            JOIN teaches AS T ON I.ID = T.ID
            JOIN takes AS TA ON TA.course_id = T.course_id AND TA.sec_id = T.sec_id AND TA.semester = T.semester AND TA.year = T.year
            GROUP BY I.ID) AS subquery
    );
    '''
)

for x in mycursor:
    print(x)

('113', 'Sunil Agarwal')


6. Find the names of any classes (identified by course_id and section) being taught in Fall '23 such that the number of students taking that class exceeds the capacity of the classroom in which the class is taught.

In [8]:
mycursor = mydb.cursor()
mycursor.execute(
    '''
    SELECT S.course_id, S.sec_id
    FROM section S
    JOIN classroom C ON S.building = C.building AND S.room_no = C.room_no
    WHERE S.semester = 'Fall' AND S.year = 2023
    AND (SELECT COUNT(T.ID)
        FROM takes T
        WHERE T.course_id = S.course_id AND T.sec_id = S.sec_id AND T.semester = S.semester AND T.year = S.year)
        > C.capacity;
    '''
)

for x in mycursor:
    print(x)